In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

Using TensorFlow backend.


In [2]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

aug = ImageDataGenerator(rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
model.fit_generator(aug.flow(x_train, y_train, batch_size=batch_size),validation_data=(x_test, y_test), steps_per_epoch=len(x_train) // batch_size,
epochs=epochs)
#history = model.fit(x_train, y_train,
#                    batch_size=batch_size,
#                    epochs=epochs,
#                    verbose=1,
#                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)       

In [4]:
#rotation_range改成30
aug = ImageDataGenerator(rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
model.fit_generator(aug.flow(x_train, y_train, batch_size=batch_size),validation_data=(x_test, y_test), steps_per_epoch=len(x_train) // batch_size,
epochs=epochs)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
390/390 [==============================] - 28s 71ms/step - loss: 1.0910 - accuracy: 0.6169 - val_loss: 0.8675 - val_accuracy: 0.6999
Epoch 2/10
390/390 [==============================] - 26s 68ms/step - loss: 1.0698 - accuracy: 0.6258 - val_loss: 0.8804 - val_accuracy: 0.7033
Epoch 3/10
390/390 [==============================] - 27s 68ms/step - loss: 1.0518 - accuracy: 0.6334 - val_loss: 0.9824 - val_accuracy: 0.6711
Epoch 4/10
390/390 [==============================] - 26s 68ms/step - loss: 1.0417 - accuracy: 0.6392 - val_loss: 1.0433 - val_accuracy: 0.6635
Epoch 5/10
390/390 [==============================] - 26s 67ms/step - loss: 1.0291 - accuracy: 0.6437 - val_loss: 0.8700 - val_accuracy: 0.7084
Epoch 6/10
390/390 [==============================] - 26s 67ms/step - loss: 1.0215 - accuracy: 0.6476 - val_loss: 0.8723 - val_accuracy: 0.7051
Epoch 7/10
390/390 [==============================] - 26s 67ms/step - loss: 1.0075 - accuracy: 0.6524 - val_loss: 0.8836 - val_accuracy:

> 不同參數調整會影响預測準確度